# Loading terrain features
Maps are generated randomly,however attributes are described via a spreadsheet. 
https://docs.google.com/spreadsheets/d/1E1y6QdK3IhzzoEjvlLrc81D0_3qJxtYZpoouhvit4RI/edit?usp=sharing

**These are the key attribute tables**<br>
* **T**errain **D**etail (ddf) = Information for each area. Which features, which monsters, the probability of what events. 
* **T**errain **I**tems** D**ata **F**rame (TIDF) = Properties of a terrain feature, its color, how it is rendered, and what happens when the user interacts with that object
* **B**estiary (bdf) = properties of creatures, general stats and attach methods. 

In [17]:
import pandas as pd
import numpy as np
import boto
import yaml

In [18]:
from boto.s3.connection import S3Connection

myKeys = yaml.load(open('/home/billmanh/Downloads/Key_file', 'r'))
AWSSecretKey=myKeys['AWSSecretKey']
AWSAccessKeyId=myKeys['AWSAccessKeyId']
conn = S3Connection(AWSAccessKeyId, AWSSecretKey)
mybucket = conn.get_bucket('flaskgame')

myKey = mybucket.get_key("maps/william.jeffrey.harding@gmail.comterrainMap.json")
myKey

<Key: flaskgame,maps/william.jeffrey.harding@gmail.comterrainMap.json>

# Loading terrain data and manipulating locally

Loading data from TSV files and prepping them for import to boto

In [19]:
ddf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Terrain Detail.tsv",sep="\t",index_col=0)
tidf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Terrain Items.tsv",sep="\t",index_col=0)
bdf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Bestiary.tsv",sep="\t",index_col=0)

In [20]:
t_details = yaml.load(ddf.T.to_json())


print(t_details.keys())
print(t_details.get('ds'))

dict_keys(['fo', 'sw', 'mt', 'sa', 'pl', 'ds', 'ic', 'oa'])
{'Type Name': 'desert', 'Type': 'normal', 'Danger Level': 1, 'Creature Probability': 0.1, 'Treasure Level': 1, 'Description': 'a barren desert', 'Detailed Description': None, 'creatures': 'scorpion', 'Terrain Textures': "[         {'name':'Sand',         'abundance':300,         'density':5} ]"}


In [21]:
#wierd issue causes the dict to load as string. Fixed it.
ddf['Terrain Textures'] = ddf['Terrain Textures'].apply(lambda x: yaml.load(x))
ddf['Terrain Textures']

Code
fo    [{'name': 'Tree', 'density': 6, 'abundance': 1...
sw    [{'name': 'Fern', 'abundance': 5, 'density': 5...
mt    [{'name': 'Boulder', 'density': 6, 'abundance'...
sa    [{'name': 'Umbrella Thorn', 'density': 6, 'abu...
pl    [{'name': 'Grass', 'abundance': 150, 'density'...
ds    [{'name': 'Sand', 'abundance': 300, 'density':...
ic    [{'name': 'Glacier', 'density': 6, 'abundance'...
oa    [{'name': 'Palm Tree', 'density': 6, 'abundanc...
Name: Terrain Textures, dtype: object

In [22]:
type(ddf['Terrain Textures'][1])

list

In [23]:
ddf.head()

,Type Name,Type,Danger Level,Creature Probability,Treasure Level,Description,Detailed Description,creatures,Terrain Textures
Code,,,,,,,,,
fo,forest,normal,4,0.5,3,a quiet deciduous forest,NaN,goblin,"[{'name': 'Tree', 'density': 6, 'abundance': 1..."
sw,swamp,normal,8,0.8,2,a desolate swamp,NaN,goblin,"[{'name': 'Fern', 'abundance': 5, 'density': 5..."
mt,mountain,normal,6,0.4,5,a rugged mountain range,NaN,goblin,"[{'name': 'Boulder', 'density': 6, 'abundance'..."
sa,savanna,normal,3,0.1,3,a dry savana,NaN,tiger,"[{'name': 'Umbrella Thorn', 'density': 6, 'abu..."
pl,plain,normal,2,0.1,2,a grassy plain,NaN,tiger,"[{'name': 'Grass', 'abundance': 150, 'density'..."


In [24]:
t_items = tidf.T.to_dict()
t_items['Boulder']

{'affect': 'bump',
 'affectAmt': 2,
 'affectText': '*** hit a boulder',
 'hex': '#D3D3D3',
 'size': 10,
 'spread': 'line'}

In [25]:
tidf

,hex,size,spread,affect,affectAmt,affectText
name,,,,,,
Tree,#228B22,20,jitter,bump,2,*** hit a tree
Bush,#006400,10,jitter,hide,3,*** is hiding in a bush
Fern,#006400,5,jitter,no,0,none
Mud Pit,#794c13,10,jitter,slow,9,*** is stuck in the mud
Boulder,#D3D3D3,10,line,bump,2,*** hit a boulder
Stone,#000000,2,scatter,no,0,none
Rock,#2F4F4F,5,scatter,bump,2,*** hit a rock
Umbrella Thorn,#b39956,20,scatter,bump,1,*** hit an Umbrella Thorn tree
Tall grass,#fbf9de,1,scatter,no,0,none


In [26]:
beastiary = bdf.T.to_dict()

print(beastiary.keys())
print(beastiary['goblin'])

dict_keys(['goblin', 'tiger', 'scorpion', 'yeti', 'raider'])
{'name': 'Goblin', 'health': 3, 'healthMaxVariance': 2, 'move': 5, 'size': 5, 'move type': 'swarm', 'render type': 'pack', 'group min': 5, 'group max': 10, 'color': '#3E8E3E', 'perception': 10, 'attackType': 'swung a rusty sword', 'damage': 4}


In [27]:
bdf.head()

,name,health,healthMaxVariance,move,size,move type,render type,group min,group max,color,perception,attackType,damage
index,,,,,,,,,,,,,
goblin,Goblin,3,2,5,5,swarm,pack,5,10,#3E8E3E,10,swung a rusty sword,4
tiger,Tiger,8,4,10,6,swarm,alone,1,1,#e08d3c,200,swung it's might claws,12
scorpion,Giant Scorpion,8,7,3,8,swarm,scatter,1,2,#716D6F,10,struck with it's stinger,12
yeti,Yeti,15,3,5,15,swarm,alone,1,1,#a0d6b4,50,swung with it's fists,8
raider,Raider,15,5,6,10,swarm,pack,2,3,#4a3b29,150,swung a daggar,6


## Transform data into useful types
any cleaning steps taken here

In [77]:
[yaml.load(t_details[list(t_details.keys())[1]]['Terrain Textures']) for i in list(t_details.keys())]

[[{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}],
 [{'abundance': 5, 'density': 5, 'name': 'Fern'},
  {'abundance': 15, 'density': 5, 'name': 'Mud Pit'}]]

In [82]:
for i in list(t_details.keys()):
    t_details[i]['Terrain Textures'] = yaml.load(t_details[i]['Terrain Textures'])

In [83]:
t_details[i]['Terrain Textures']

[{'abundance': 10, 'density': 6, 'name': 'Palm Tree'},
 {'abundance': 150, 'density': 5, 'name': 'Sand'}]

In [84]:
type(t_details[list(t_details.keys())[1]])

dict

## Add Terrain Items to Terrain Details
I'm adding terrain textures to the terrain details because those features are generally small. If this list gets really large I might load the terrain on demand.

In [86]:
for item in t_details.keys():
    print(t_details[item]['Type Name'])
    for ter in t_details[item]['Terrain Textures']:
        ter['hex'] = tidf.ix[ter['name'],'hex']
        ter['size'] = tidf.ix[ter['name'],'size']
        ter['spread'] = tidf.ix[ter['name'],'spread']
        ter['affect'] = tidf.ix[ter['name'],'affect']
        ter['affectAmt'] = tidf.ix[ter['name'],'affectAmt']
        ter['affectText'] = tidf.ix[ter['name'],'affectText']

forest
swamp
mountain
savanna
plain
desert
tundra
oasis


In [87]:
ter

{'abundance': 150,
 'affect': 'no',
 'affectAmt': 0,
 'affectText': 'none',
 'density': 5,
 'hex': '#c2b280',
 'name': 'Sand',
 'size': 1,
 'spread': 'scatter'}

In [88]:
t_details

{'ds': {'Creature Probability': 0.1,
  'Danger Level': 1,
  'Description': 'a barren desert',
  'Detailed Description': None,
  'Terrain Textures': [{'abundance': 300,
    'affect': 'no',
    'affectAmt': 0,
    'affectText': 'none',
    'density': 5,
    'hex': '#c2b280',
    'name': 'Sand',
    'size': 1,
    'spread': 'scatter'}],
  'Treasure Level': 1,
  'Type': 'normal',
  'Type Name': 'desert',
  'creatures': 'scorpion'},
 'fo': {'Creature Probability': 0.5,
  'Danger Level': 4,
  'Description': 'a quiet deciduous forest',
  'Detailed Description': None,
  'Terrain Textures': [{'abundance': 10,
    'affect': 'bump',
    'affectAmt': 2,
    'affectText': '*** hit a tree',
    'density': 6,
    'hex': '#228B22',
    'name': 'Tree',
    'size': 20,
    'spread': 'jitter'},
   {'abundance': 5,
    'affect': 'hide',
    'affectAmt': 3,
    'affectText': '*** is hiding in a bush',
    'density': 5,
    'hex': '#006400',
    'name': 'Bush',
    'size': 10,
    'spread': 'jitter'},
   {'

In [89]:
print(t_details.keys())
print(t_details.get('pl'))

dict_keys(['fo', 'sw', 'mt', 'sa', 'pl', 'ds', 'ic', 'oa'])
{'Type Name': 'plain', 'Type': 'normal', 'Danger Level': 2, 'Creature Probability': 0.1, 'Treasure Level': 2, 'Description': 'a grassy plain', 'Detailed Description': None, 'creatures': 'tiger', 'Terrain Textures': [{'name': 'Grass', 'abundance': 150, 'density': 5, 'hex': '#006400', 'size': 1, 'spread': 'scatter', 'affect': 'no', 'affectAmt': 0, 'affectText': 'none'}, {'name': 'Rock', 'abundance': 5, 'density': 5, 'hex': '#2F4F4F', 'size': 5, 'spread': 'scatter', 'affect': 'bump', 'affectAmt': 2, 'affectText': '*** hit a rock'}]}


## Saving terrain Detail to JSON and sending them to S3 via BOTO

Each terrain detail item is saved as a separate file so that not all terrain types need to be loaded at the same time. 

In [90]:
from boto.s3.key import Key

for item in t_details.keys():
    myKey = mybucket.get_key('terrain/' + item)
    if myKey==None:
        print("Saving key: " + item)
        k = Key(mybucket)
        k.key = 'terrain/' + item
        k.set_contents_from_string(str(t_details[item]))
    else:
        print(item+" already exists... Overwriting")
        myKey.set_contents_from_string(str(t_details[item]))

fo already exists... Overwriting
sw already exists... Overwriting
mt already exists... Overwriting
sa already exists... Overwriting
pl already exists... Overwriting
ds already exists... Overwriting
ic already exists... Overwriting
oa already exists... Overwriting


In [91]:
# Uploading all of t_details for use when creating worlds
k = Key(mybucket)
k.key = 'terrain/' + 'terrain_details'
k.set_contents_from_string(str(t_details))

4454

In [92]:
# Uploading the Bestiary
k = Key(mybucket)
k.key = 'terrain/' + 'beastiary'
k.set_contents_from_string(str(beastiary))

1294

## working inside of terrain textures

In [43]:
terrData = yaml.load(myKey.get_contents_as_string())

In [44]:
terrData

{'Creature Probability': 0.3,
 'Danger Level': 5,
 'Description': 'an oasis',
 'Detailed Description': 'None',
 'Terrain Textures': "[         {'name':'Palm Tree',         'density':6,         'abundance':10},         {'name':'Sand',         'abundance':150,         'density':5} ]",
 'Treasure Level': 5,
 'Type': 'special',
 'Type Name': 'oasis',
 'creatures': 'raider'}